In [1]:
#event of clicking the button
import rospy
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import Odometry
from move_base_msgs.msg import MoveBaseActionGoal
from move_base_msgs.msg import MoveBaseActionFeedback
from actionlib_msgs.msg import GoalStatusArray
from actionlib_msgs.msg import GoalID

import array as arr
import numpy as np
import time

#libraries for the widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, FloatText
import ipywidgets as widgets
from IPython.display import display

#libraries for plotting
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import rospy
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
import math
%matplotlib widget

DTH = 1.0
GOAL_TH = 0.5

#declare the boolean variable for differentiating the three modalities
manDrive = False
collisionAvoidence = False
goal = False


my_vel = Twist() #variable of type msg Twist
goal_msg = MoveBaseActionGoal() #variable of type msg for storing the goal to publish
canc_goal = GoalID() #message fro cancelling a goal when is reached
ID = ""
reached = 0
not_reached = 0

#variables for computing the goal reached
time_start = 0
time_end = 0
time_passed = 0

#textBox for the coordinate x of the goal
textBox_x = widgets.FloatText(
    value = 0,
    description = 'type a x:',
    disabled = False
)

#textBox for the coordinate y of the goal
textBox_y = widgets.FloatText(
    value = 0,
    description = 'type a y:',
    disabled = False
)

In [2]:
#function for cancelling the goal once it is reached
def CancelGoal():
    global goal
    global canc_goal
    global ID
    global pub_cancel
    
    if goal == False:
        return False
    canc_goal.id = ID
    pub_cancel.publish(canc_goal)
    goal = False
    return True


In [3]:
#function called everytime the current position of the robot is changing
def CurrentPositionCallback(msg):
    global ID
    global goal
    global textBox_x
    global textBox_y
    global reached
    global not_reached
    global time_start
    global time_end
    global time_passed 
    TIMEOUT = 180
    
    #Update the goal ID if there is a new goal
    if ID != msg.status.goal_id.id:
    
        ID = msg.status.goal_id.id
    
    
    if goal == True:
        #take the current robot position
        robot_x = msg.feedback.base_position.pose.position.x
        robot_y = msg.feedback.base_position.pose.position.y
    
        #calculate the error from the actual position and the goal position
        dist_x = robot_x - textBox_x.value 
        dist_y = robot_y - textBox_y.value 

        #if the robot is on the goal position
        if abs(dist_x) <= 0.5 and abs(dist_y) <= 0.5:
    
            print("Goal reached")
            reached += 1
            print("reached", reached)
            time_passed = -1
            
            CancelGoal()
         
        time_end = time.time_ns()/ (10 ** 9)
        time_passed = (time_end - time_start)
        
        #if the time passed is over 3 minutes the goal is considered not reacheable
        if(time_passed > TIMEOUT):
            print("The actual goal can't be reached!\n")
            not_reached += 1
            print("not_reached:", not_reached)
            CancelGoal() #Cancel the goal if it can't be reached
            time_passed = -1
        

In [4]:
#function called everytime that cmd_vel topic is changed
def GetVelCallback(msg): 
    global manDrive
    global collisionAvoidence
    
    if manDrive == False and collisionAvoidence == False:
        return
    
    if manDrive == True and collisionAvoidence == False:
        pub_vel.publish(msg)
        return
    
    if manDrive == False and collisionAvoidence == True:
        my_vel.linear.x = msg.linear.x
        my_vel.angular.z = msg.angular.z    

In [5]:
#function for computing the minimum element of an array
def ComputeMinDist(*scan, size):
    
    min_dist = np.min(scan)


    return min_dist



In [6]:
#function called everytime the third modality is chosen for enabling the collision avoidence
def CollisionAvoidenceCallback(msg):
    
    global my_vel
    global collisionAvoidence
    
    if collisionAvoidence == False:
        return
    #get array ranges by laserscan
    scan = np.tile(np.float32(0), len(msg.ranges))
    #scan = np.arange(len(msg.ranges))
    for j in range(0, len(msg.ranges)):
    
        scan[j] = msg.ranges[j]
    
    
    #divide the array ranges values in 5 sectors 
    sector_size = (len(msg.ranges))/5 
    left = np.arange(sector_size)
    front_left = np.arange(sector_size)
    front = np.arange(sector_size)
    front_right = np.arange(sector_size)
    right = np.arange(sector_size)
    
    #fill sector arrays
    for i in range(0, int(sector_size)):
    
        right[i] = scan[i]
    
    
    for i in range(0, int(sector_size)):
    
        front_right[i] = scan[i + int(sector_size)]
    
    
    for i in range(0, int(sector_size)):
    
        front[i] = scan[i + 2*int(sector_size)]
    
    
    for i in range(0, int(sector_size)):
    
        front_left[i] = scan[i + 3*int(sector_size)]
    
    
    for i in range(0, int(sector_size)):
    
        left[i] = scan[i + 4*int(sector_size)]
    

    #check if the robot is going to crash to obstacles and in case stop it
    if(ComputeMinDist(front, size = int(sector_size)) < 1.0): 
    
        if(my_vel.linear.x > 0.0 and my_vel.angular.z == 0.0):
        
            my_vel.linear.x = 0
            print("Attenction: there is an obstacle in front\n") 
        
    
    if(ComputeMinDist(front_left, size = int(sector_size)) < 1.0):
    
        if(my_vel.linear.x > 0.0 and my_vel.angular.z > 0.0):
            
            my_vel.linear.x = 0.0
            my_vel.angular.z = 0.0
            print("Attenction: there is an obstacle on the front-left\n")

    if(ComputeMinDist(front_right, size = int(sector_size)) < 1.0):

        if(my_vel.linear.x > 0.0 and my_vel.angular.z < 0.0):

            my_vel.linear.x = 0.0
            my_vel.angular.z = 0.0
            print("Attenction: there is an obstacle on the front-right\n")

    if(ComputeMinDist(left, size = int(sector_size)) < 1.0):

        if(my_vel.linear.x == 0.0 and my_vel.angular.z > 0.0):

            my_vel.angular.z = 0.0
            print("Attenction: there is an obstacle on the left\n")
            
    if(ComputeMinDist(right, size = int(sector_size)) < 1.0):

        if(my_vel.linear.x == 0.0 and my_vel.angular.z < 0.0):

            my_vel.angular.z = 0.0
            print("Attenction: there is an obstacle on the right\n")

    pub_vel.publish(my_vel) #publish the corrected velocity for avoiding collisions 

In [7]:
#function for displaying the widgets for setting the desirable goal
def DisplayDesirableGoal(): 
    
    output_textBox_goal_x = widgets.Output()
    display(textBox_x, output_textBox_goal_x)
    
    output_textBox_goal_y = widgets.Output()
    display(textBox_y, output_textBox_goal_y)

In [8]:
#function trigguered everytime 'sent goal button' is pressed
def OnButtonSendGoalClicked(b):
    global goal
    global time_start
    global time_passed
    global time_end
    
    time_passed = -1
    time_end = time.time_ns()/ (10 ** 9)
    time_start = time.time_ns() / (10 ** 9)
    
    goal = True
    
    #setting the coordinates goal fields with the current textBox value    
    goal_msg.goal.target_pose.pose.position.x = textBox_x.value 
    goal_msg.goal.target_pose.pose.position.y = textBox_y.value
    
    #publish the goal chosen by the user
    pub_goal.publish(goal_msg)
    print("goal sent")

In [9]:
#function trigguered everytime 'button1' is pressed
def OnButton1Clicked(b):
    global manDrive
    global collisionAvoidence
    global goal
    global goal_msg
    
    print("1st modality chosen.")
        
    #set two boolean variables according to the first modality
    manDrive = False
    collisionAvoidence = False
    print("what target do you want to achieve?")
        
    DisplayDesirableGoal()
        
    #print at screen the button 'send goal'
    display(button_send_goal, output_send_goal)
    
    #default operations
    goal_msg.goal.target_pose.header.frame_id = 'map'
    goal_msg.goal.target_pose.pose.orientation.w = 1
        
    #when the button 'send goal' is pressend, execute the function 'on_ButtonSendGoalClicked'
    button_send_goal.on_click(OnButtonSendGoalClicked)   

In [10]:
#function trigguered everytime button2 is pressed
def OnButton2Clicked(b):
    global manDrive
    global collisionAvoidence
    
    print("2nd modality chosen")
    
    CancelGoal()

    #set the boolean variables according to the second modality
    manDrive = True
    collisionAvoidence = False
    
    DriveRobot()
        

In [11]:
#function trigguered everytime button3 is pressed
def OnButton3Clicked(b):
    global manDrive
    global collisionAvoidence
    
    print("3rd modality chosen")
    
    #set the boolean variables according to the third modality
    manDrive = False
    collisionAvoidence = True
    
    DriveRobot()

In [12]:
#function for driving the robot through a keyboard
def DriveRobot():
    
    button5 = Button(description='left',
        layout=Layout(width='auto', align="center", grid_area='b5'),
        style=ButtonStyle(button_color='salmon'))
    button6 = Button(description='straight',
        layout=Layout(width='auto', grid_area='b6'),
        style=ButtonStyle(button_color='salmon'))
    button7 = Button(description='right',
        layout=Layout(width='auto', grid_area='b7'),
        style=ButtonStyle(button_color='salmon'))
    button8 = Button(description='back',
        layout=Layout(width='auto', grid_area='b8'),
        style=ButtonStyle(button_color='salmon'))
    button9 = Button(description='stop',
        layout=Layout(width='auto', grid_area='b9'),
        style=ButtonStyle(button_color='salmon'))
    
    output5 = widgets.Output()
    output6 = widgets.Output()
    output7 = widgets.Output()
    output8 = widgets.Output()
    output9 = widgets.Output()
    
    display(button5, output5)
    display(button6, output6)
    display(button7, output7)
    display(button8, output8)
    display(button9, output9)
    
    button5.on_click(OnButton5Clicked)
    button6.on_click(OnButton6Clicked)
    button7.on_click(OnButton7Clicked)
    button8.on_click(OnButton8Clicked)
    button9.on_click(OnButton9Clicked)

In [13]:
#function executed everytime the button5 is typed
def OnButton5Clicked(b):
    #turn left
    global my_vel
    
    my_vel.linear.x = 0.5
    my_vel.angular.z = 1.0
    
    pub_vel.publish(my_vel)

#function executed everytime the button6 is typed
def OnButton6Clicked(b):
    #go straight on
    global my_vel
    
    my_vel.linear.x = 0.5
    my_vel.angular.z = 0.0
    
    pub_vel.publish(my_vel)

#function executed everytime the button7 is typed
def OnButton7Clicked(b):
    #turn right
    global my_vel
    
    my_vel.linear.x = 0.5
    my_vel.angular.z = -1.0
    
    pub_vel.publish(my_vel)

#function executed everytime the button8 is typed
def OnButton8Clicked(b):
    #go back
    global my_vel
    
    my_vel.linear.x = -0.5
    my_vel.angular.z = 0.0
    
    pub_vel.publish(my_vel)

#function executed everytime the button9 is typed
def OnButton9Clicked(b):
    #stop
    global my_vel
    
    my_vel.linear.x = 0.0
    my_vel.angular.z = 0.0
    
    pub_vel.publish(my_vel)

In [14]:
rospy.init_node("continuation_assignment")

#publisher
pub_vel = rospy.Publisher("/cmd_vel", Twist, queue_size=1)
pub_goal = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=1)
pub_cancel = rospy.Publisher("/move_base/cancel", GoalID, queue_size=1)

#subscriber
sub_vel = rospy.Subscriber("/new_cmd_vel", Twist, GetVelCallback)
sub_pos = rospy.Subscriber("/move_base/feedback", MoveBaseActionFeedback, CurrentPositionCallback)
sub_laser = rospy.Subscriber("/scan", LaserScan, CollisionAvoidenceCallback)

print("select a modality")

#definition of the 3 modalities buttons widgets
button1 = widgets.Button(description = "modality 1")
button2 = widgets.Button(description = "modality 2")
button3 = widgets.Button(description = "modality 3")
button4 = widgets.Button(description = "view goal statistics",
                        style=ButtonStyle(button_color='moccasin'))
output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()
output4 = widgets.Output()

#print at screen the 3 modalities buttons
display(button1, output1)
display(button2, output2)
display(button3, output3)
display(button4, output4)

#definition of the button for sending the goal chosen by the user
button_send_goal = widgets.Button(description="send goal")
output_send_goal = widgets.Output()

select a modality


Button(description='modality 1', style=ButtonStyle())

Output()

Button(description='modality 2', style=ButtonStyle())

Output()

...

Output()

Button(description='back', layout=Layout(grid_area='b8', width='auto'), style=ButtonStyle(button_color='salmon…

Output()

Button(description='stop', layout=Layout(grid_area='b9', width='auto'), style=ButtonStyle(button_color='salmon…

Output()

In [15]:
#class for plotting the laser scanner
class VisualiseObstacles:
    
    def __init__(self):
        
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [],'ro')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        self.ax.set_xlim(0, 720)
        self.ax.set_ylim(0, 14)
        return self.ln

    def LaserScanCallback(self, msg):
        self.y_data = []
        self.x_data = []
        for i in range(0, len(msg.ranges), 10):
            
            self.y_data.append(msg.ranges[i])
            self.x_data.append(i)
    
    def update_plot(self, frame): 
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

vis = VisualiseObstacles()
sub = rospy.Subscriber("/scan", LaserScan, vis.LaserScanCallback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
#class for plotting the robot position
class VisualizeOdometry:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [],'ro')
        self.x_data, self.y_data = [] , []
        
    def PlotInit(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def OdomCallback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
    def UpdatePlot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
vis2 = VisualizeOdometry()
sub2 = rospy.Subscriber('/odom', Odometry, vis2.OdomCallback)

ani2 = FuncAnimation(vis2.fig, vis2.UpdatePlot, init_func=vis2.PlotInit)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig, ax = plt.subplots()

#function for plotting the situation of goal reached/not reached
def OnButton4Clicked(b):
    global fig, ax
    ax.cla()
    goals_reached = (reached)
    goals_not_reached = (not_reached)
    index = np.arange(1)
    eConf = {'ecolor': '0.3'}
    ax.bar(index, goals_reached, align='edge', width=0.1, alpha=0.4,
    color='b', error_kw=eConf, label='reached goal')
    ax.bar(index-0.1, goals_not_reached, align='edge', width=0.1, alpha=0.4,
    color='r', error_kw=eConf, label='not reached goal')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title('number of reached vs not reached goals')
    ax.set_xticks(index + 0.35, (''))
    ax.legend()
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
button1.on_click(OnButton1Clicked)
button2.on_click(OnButton2Clicked)
button3.on_click(OnButton3Clicked)
button4.on_click(OnButton4Clicked)